# Audio Classification: Zero-Shot & Fine-Tuning

Два подхода к классификации аудио:
1. **Zero-Shot** - классификация без обучения с помощью CLAP
2. **Fine-Tuning** - дообучение wav2vec2 / HuBERT на своих данных

Применимо для:
- Классификация звуков
- Распознавание эмоций
- Музыкальные жанры
- Sound event detection

In [ ]:
!pip install transformers torch torchaudio librosa soundfile pandas numpy scikit-learn datasets -q

In [ ]:
import torch
import torchaudio
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
from transformers import (
    AutoFeatureExtractor, 
    AutoModelForAudioClassification,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат: CSV с колонками 'file_path', 'label'
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

SAMPLE_RATE = 16000  # Wav2Vec2 работает на 16kHz
MAX_DURATION = 10  # секунд

# Label encoding
le = LabelEncoder()
train_df['label_encoded'] = le.fit_transform(train_df['label'])
num_labels = len(le.classes_)

print(f"Train samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Количество классов: {num_labels}")
print(f"Классы: {le.classes_}")
print(f"\nРаспределение классов:")
print(train_df['label'].value_counts())

## ЧАСТЬ 1: ZERO-SHOT CLASSIFICATION с CLAP

### 1.1 Загрузка CLAP модели

In [ ]:
# CLAP (Contrastive Language-Audio Pretraining)
# Альтернатива: используем модель, которая понимает текстовые описания звуков

# Для zero-shot нужна модель типа CLAP или AudioCLIP
# К сожалению, готовых моделей в HuggingFace мало
# Используем Wav2Vec2 + текстовые эмбеддинги как workaround

print("\n⚠️ CLAP модели пока ограничены в HuggingFace")
print("Для настоящего zero-shot используйте:")
print("- microsoft/CLAP (если доступна)")
print("- laion/clap-htsat-unfused")
print("\nПереходим к Fine-Tuning подходу...")

### 1.2 Zero-Shot с предобученной моделью (альтернатива)

In [ ]:
# Используем предобученную модель для быстрого baseline
# Например, модель обученная на AudioSet или ESC-50

zero_shot_model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"  # Audio Spectrogram Transformer

try:
    zero_shot_model = AutoModelForAudioClassification.from_pretrained(zero_shot_model_name)
    zero_shot_processor = AutoFeatureExtractor.from_pretrained(zero_shot_model_name)
    
    print(f"✓ Zero-shot модель загружена: {zero_shot_model_name}")
    
    # Пример предсказания
    def zero_shot_predict(audio_path):
        audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=MAX_DURATION)
        inputs = zero_shot_processor(audio, sampling_rate=SAMPLE_RATE, return_tensors="pt")
        
        with torch.no_grad():
            logits = zero_shot_model(**inputs).logits
        
        predicted_class_id = logits.argmax(-1).item()
        predicted_label = zero_shot_model.config.id2label[predicted_class_id]
        
        return predicted_label, logits
    
    # Тестируем на первом примере
    sample_audio = train_df.iloc[0]['file_path']
    pred_label, logits = zero_shot_predict(sample_audio)
    print(f"\nПример zero-shot предсказания:")
    print(f"Файл: {sample_audio}")
    print(f"Предсказанный класс: {pred_label}")
    
except Exception as e:
    print(f"⚠️ Не удалось загрузить zero-shot модель: {e}")
    print("Переходим к fine-tuning...")

## ЧАСТЬ 2: FINE-TUNING Wav2Vec2

### 2.1 Загрузка предобученной модели

In [ ]:
# Выбор модели
MODEL_NAME = "facebook/wav2vec2-base"  # базовая модель
# MODEL_NAME = "facebook/wav2vec2-large-xlsr-53"  # мультиязычная, более мощная
# MODEL_NAME = "microsoft/wavlm-base"  # альтернатива
# MODEL_NAME = "facebook/hubert-base-ls960"  # HuBERT

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

print(f"✓ Модель загружена: {MODEL_NAME}")
print(f"Параметры модели: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")

### 2.2 Подготовка данных для Trainer

In [ ]:
# Train/Val split
train_df_split, val_df_split = train_test_split(
    train_df, test_size=0.15, random_state=42, stratify=train_df['label']
)

print(f"Train: {len(train_df_split)}, Val: {len(val_df_split)}")

# Создание HuggingFace Dataset
def create_dataset(df):
    dataset_dict = {
        'audio': df['file_path'].tolist(),
        'label': df['label_encoded'].tolist()
    }
    dataset = Dataset.from_dict(dataset_dict)
    dataset = dataset.cast_column('audio', Audio(sampling_rate=SAMPLE_RATE))
    return dataset

train_dataset = create_dataset(train_df_split)
val_dataset = create_dataset(val_df_split)

print(f"\n✓ Datasets созданы!")
print(f"Train dataset: {train_dataset}")
print(f"Val dataset: {val_dataset}")

### 2.3 Preprocessing функция

In [ ]:
def preprocess_function(examples):
    """
    Preprocessing для аудио
    """
    audio_arrays = [x["array"] for x in examples["audio"]]
    
    # Обрезаем или дополняем до MAX_DURATION
    max_length = int(SAMPLE_RATE * MAX_DURATION)
    processed_arrays = []
    
    for audio in audio_arrays:
        if len(audio) > max_length:
            audio = audio[:max_length]
        elif len(audio) < max_length:
            audio = np.pad(audio, (0, max_length - len(audio)), mode='constant')
        processed_arrays.append(audio)
    
    # Feature extraction
    inputs = feature_extractor(
        processed_arrays,
        sampling_rate=SAMPLE_RATE,
        return_tensors="pt",
        padding=True
    )
    
    inputs["labels"] = examples["label"]
    return inputs

# Применяем preprocessing
print("Preprocessing данных...")
train_dataset = train_dataset.map(preprocess_function, batched=True, batch_size=8, remove_columns=["audio"])
val_dataset = val_dataset.map(preprocess_function, batched=True, batch_size=8, remove_columns=["audio"])

print("✓ Preprocessing завершен!")

### 2.4 Metrics

In [ ]:
def compute_metrics(eval_pred):
    """
    Вычисление метрик
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    
    return {
        'accuracy': accuracy,
        'f1': f1
    }

### 2.5 Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./audio_finetuned_model",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=50,
    eval_steps=100,
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    fp16=torch.cuda.is_available(),  # Mixed precision training
    dataloader_num_workers=2,
)

print("✓ Training arguments настроены!")

### 2.6 Fine-Tuning

In [ ]:
# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("\nНачало fine-tuning...\n")
print("="*60)

# Обучение
train_result = trainer.train()

print("\n" + "="*60)
print("✓ Fine-tuning завершен!")
print(f"Training loss: {train_result.training_loss:.4f}")
print("="*60)

### 2.7 Оценка на валидации

In [ ]:
# Evaluation
eval_results = trainer.evaluate()

print("\nРезультаты на валидации:")
print("="*60)
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")
print("="*60)

### 2.8 Детальный анализ

In [ ]:
# Предсказания на валидации
predictions = trainer.predict(val_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

# Confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 3. Предсказания на Test

In [ ]:
# Подготовка test dataset
test_df_copy = test_df.copy()
test_df_copy['label_encoded'] = 0  # Dummy label

test_dataset = create_dataset(test_df_copy)
test_dataset = test_dataset.map(preprocess_function, batched=True, batch_size=8, remove_columns=["audio"])

print(f"✓ Test dataset готов: {len(test_dataset)} samples")

# Предсказания
print("\nГенерация предсказаний...")
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)

# Декодирование меток
test_pred_labels_decoded = le.inverse_transform(test_pred_labels)

print(f"\n✓ Предсказания готовы!")
print(f"\nРаспределение предсказанных классов:")
print(pd.Series(test_pred_labels_decoded).value_counts())

## 4. Сохранение модели

In [ ]:
# Сохранение лучшей модели
trainer.save_model("./best_audio_model")
feature_extractor.save_pretrained("./best_audio_model")

print("✓ Модель сохранена в ./best_audio_model")

## 5. Inference функция

In [ ]:
def predict_audio(audio_path, model, feature_extractor, label_encoder):
    """
    Предсказание класса для одного аудио файла
    """
    # Загрузка аудио
    audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=MAX_DURATION)
    
    # Padding
    max_length = int(SAMPLE_RATE * MAX_DURATION)
    if len(audio) < max_length:
        audio = np.pad(audio, (0, max_length - len(audio)), mode='constant')
    elif len(audio) > max_length:
        audio = audio[:max_length]
    
    # Feature extraction
    inputs = feature_extractor(
        audio,
        sampling_rate=SAMPLE_RATE,
        return_tensors="pt"
    )
    
    # Предсказание
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class_id = logits.argmax(-1).item()
    predicted_label = label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = torch.softmax(logits, dim=-1)[0][predicted_class_id].item()
    
    return predicted_label, confidence

# Пример использования
sample_file = test_df.iloc[0]['file_path']
pred_label, confidence = predict_audio(sample_file, model, feature_extractor, le)
print(f"\nПример предсказания:")
print(f"Файл: {sample_file}")
print(f"Класс: {pred_label}")
print(f"Уверенность: {confidence:.4f}")

## 6. Submission

In [ ]:
submission = pd.DataFrame({
    'id': test_df.index,  # или test_df['id']
    'prediction': test_pred_labels_decoded
})

submission.to_csv('audio_classification_submission.csv', index=False)
print("\n✓ Submission сохранен!")
print(submission.head(10))